In [11]:
import pandas as pd
import numpy as np
import json
import os
import re

In [12]:
def extract_sentences(data, segment, isprint):
    """
    parse Json to get unspecified field count
    
    Args:
        data (string): String of properly formatted JSON.

    Returns:
        
    """
    def allindices(string, sub, offset=0):
        listindex = []
        i = string.find(sub, offset)
        while i >= 0:
            listindex.append(i)
            i = string.find(sub, i + 1)
        return listindex
#     print("Segment:", segment)
    left_fullstop = -10000
    left_break_tag = -10000
    right_fullstop = 10000
    right_break_tag = 10000


    collated_sentences = []
    json_data = json.loads(data)
    for attribute, value in json_data.items():
        endIndexInSegment = value["endIndexInSegment"]-1
        if isprint:
            print("endIndexInSegment:", endIndexInSegment)
        startIndexInSegment = value["startIndexInSegment"]
        if isprint:
            print("startIndexInSegment:", startIndexInSegment)

        json_value = value["value"]
        if json_value == "Unspecified":
            continue
        else:
            if startIndexInSegment == 0 and endIndexInSegment == len(segment):
                collated_sentences.append(segment[startIndexInSegment:endIndexInSegment-1].strip())
                continue
            if startIndexInSegment == 0:
                left_fullstop = 0
                left_break_tag = 0
            if endIndexInSegment == len(segment):
                right_fullstop = 0
                right_break_tag = 0
            
            
#             print("value: ", value)
            full_stops = allindices(segment, ".")
            break_tags = allindices(segment, "<br>")
            if isprint:
                print("full_stops: ", full_stops)
            if isprint:
                print("break_tags: ", break_tags)
            left_fullstops = [x for x in full_stops if x <= startIndexInSegment]
            right_fullstops = [x for x in full_stops if x >= endIndexInSegment]
            left_break_tags = [x for x in break_tags if x <= startIndexInSegment]
            right_break_tags = [x for x in break_tags if x >= endIndexInSegment]
            
            if len(left_fullstops) > 1: 
                if left_fullstop is not 0:
                    left_fullstop = max(left_fullstops)+1
            else:
                if len(left_fullstops) == 1:
                    if left_fullstop is not 0:
                        left_fullstop = left_fullstops[0]+1
#                 else:
#                     if len(left_fullstops == 0):
#                         left_fullstop = 0
            

            if len(right_fullstops) > 1: 
                if right_fullstops is not 0:
                    right_fullstop = min(right_fullstops)+1
            else:
                if len(right_fullstops) == 1:
                    if right_fullstop is not 0:
                        right_fullstop = right_fullstops[0]+1


                        
            if len(left_break_tags) > 1: 
                if left_break_tag is not 0:
                    left_break_tag = max(left_break_tags)+4
            else:
                if len(left_break_tags) == 1:
                    if left_break_tag is not 0:
                        left_break_tag = left_break_tags[0]+4
#                 else:
#                     if len(left_fullstops == 0):
#                         left_fullstop = 0





            if len(right_break_tags) > 1: 
                if right_break_tag is not 0:
                    right_break_tag = min(right_break_tags)+4
            else:
                if len(right_break_tags) == 1:
                    if right_break_tag is not 0:
                        right_break_tag = right_break_tags[0]+4
#                 else:
#                     if len(left_fullstops == 0):
#                         left_fullstop = 0

            if isprint:
                print("right_fullstop: ", right_fullstop)
            if isprint:
                print("right_break_tag: ", right_break_tag)
            left = max(left_fullstop, left_break_tag)
            right = min(right_fullstop, right_break_tag)
            if isprint:
                print("Collated segment:", segment[left:right].strip())
            collated_sentences.append(segment[left:right].strip())

    return collated_sentences

In [13]:
def create_df():
    """
    parse Json to get unspecified field count
    
    Args:
        data (string): String of properly formatted JSON.

    Returns:
        
    """
    dictionary = {}
    path_sanitized_policies = "/Users/bhramoss/Documents/CMU/sem2/PDS/pds_project/MineB4USign/OPP-115/sanitized_policies/"
    path_annotations = "/Users/bhramoss/Documents/CMU/sem2/PDS/pds_project/MineB4USign/OPP-115/annotations/"
    regex_html = "(.*).html"
    regex_csv = "(.*).csv"
    for filename_sanitized_policies in os.listdir(path_sanitized_policies):
        if filename_sanitized_policies == ".DS_Store":
            continue
        m = re.match(regex_html, filename_sanitized_policies)
        groups = m.groups()
        filename_sanitized_policies_no_extension = groups[0]
#         print("filename: ", filename_sanitized_policies)
        path_sanitized_policies_file = path_sanitized_policies + "/" + filename_sanitized_policies 
#         file_list = os.listdir(path_annotations)
        f = open(path_sanitized_policies_file, "r")
        content = f.read()
        segments = content.split("|||")
        path_annotations_file = path_annotations + filename_sanitized_policies_no_extension + ".csv"
#         print("path_annotations_file: ", path_annotations_file)
        annotations = pd.read_csv(path_annotations_file, keep_default_na = False, names = ["annotation ID", "batch ID", "annotator ID", "policy ID", "segment ID", "category name", "attribute-value", "policy URL", "date"]) 
        annotations = pd.concat([annotations, pd.get_dummies(annotations['category name'])], axis=1)
#         adding the segments to the dataframe.
        sLength = len(annotations['annotation ID'])
        annotations['segment'] = pd.Series(np.random.randn(sLength), index=annotations.index)
        annotations['segment'] = annotations['segment'].astype('str')
        
        annotations['collated_sentences'] = pd.Series(np.random.randn(sLength), index=annotations.index)
        annotations['collated_sentences'] = annotations['segment'].astype('str')

        count = 1
        for i, row in annotations.iterrows():
            segment_id = row['segment ID']
            annotations.at[i, 'segment'] = segments[segment_id]
#             print("annotations.at[i, 'attribute-value']", annotations.at[i, 'attribute-value'])
#             print("segments[segment_id]: ", segments[segment_id])
            count +=1
            istrue = False
            if count == 8:
                istrue = True
            collated_sentences = extract_sentences(annotations.at[i, 'attribute-value'], segments[segment_id], istrue)
            if count == 8:
                print("segments[segment_id]:", segments[segment_id])
#                 print("collated_sentences: ", collated_sentences)
            annotations.at[i, 'collated_sentences'] = collated_sentences
        dictionary[filename_sanitized_policies_no_extension] = annotations
    return dictionary
        
        
create_df()

endIndexInSegment: -2
startIndexInSegment: -1
endIndexInSegment: 449
startIndexInSegment: 335
full_stops:  [141, 227, 333, 449, 712, 1093, 1095]
break_tags:  []
right_fullstop:  450
right_break_tag:  10000
Collated segment: Your continued use of our Sites following the posting of changes to these terms will mean you accept those changes.
endIndexInSegment: 332
startIndexInSegment: 319
full_stops:  [141, 227, 333, 449, 712, 1093, 1095]
break_tags:  []
right_fullstop:  334
right_break_tag:  10000
Collated segment: We reserve the right to modify or amend the terms of our privacy policy from time to time without notice.
segments[segment_id]: By visiting our Sites, you are accepting the practices described in our privacy policy, including our use of cookies and similar online tools. If you do not agree to the terms of this privacy policy, please do not use the Sites. We reserve the right to modify or amend the terms of our privacy policy from time to time without notice. Your continued use 

{'20_theatlantic.com':      annotation ID                          batch ID  annotator ID  policy ID  \
 0             2840  test_category_labeling_highlight            84       3635   
 1             3616  test_category_labeling_highlight            82       3635   
 2             4069  test_category_labeling_highlight            88       3635   
 3             2841  test_category_labeling_highlight            84       3635   
 4             3617  test_category_labeling_highlight            82       3635   
 5             4070  test_category_labeling_highlight            88       3635   
 6             2842  test_category_labeling_highlight            84       3635   
 7             2843  test_category_labeling_highlight            84       3635   
 8             3618  test_category_labeling_highlight            82       3635   
 9             3619  test_category_labeling_highlight            82       3635   
 10            3620  test_category_labeling_highlight            82       36

Json extraction for unspecified value done below

In [10]:
def parse_api_response(data, key):
    """
    parse Json to get unspecified field count
    
    Args:
        data (string): String of properly formatted JSON.

    Returns:
        
    """
    dictionary = {}
    unspecified_count = 0
    specified_count = 0
    for row in data:
        json_data = json.loads(row)
        for attribute, value in json_data.items():
            for inner_attribute, inner_value in value.items():
#                 print("inner_attribute--->: ", inner_attribute)
#                 == is working, 'is' is not working why ?
                if inner_attribute == "value" and inner_value == "Unspecified":
                    unspecified_count += 1
                else:
                    if inner_attribute == "value":
                        specified_count += 1
    
    print("unspecified_count: ", unspecified_count)
    print("specified_count: ", specified_count)
    dictionary[key] = float(specified_count/(specified_count+unspecified_count))
    print(dictionary)

parse_api_response(annotations['attribute-value'], 'http://www.theatlantic.com/privacy-policy/')